# Langchain Day2

In [1]:
!pip install langchain --q

In [2]:
from langchain import PromptTemplate

template = """ 
You are a naming consultant for new companies.
What is a good name for a company that make {product}?

"""

prompt = PromptTemplate.from_template(template)
print(prompt.format(product = "best car"))

 
You are a naming consultant for new companies.
What is a good name for a company that make best car?




## prompt Template

In [3]:
# a prompt with no input

prompt = PromptTemplate(input_variables=[], template="Tell me a Joke." )
prompt.format()

'Tell me a Joke.'

In [4]:
# one input variable
one_input_prompt = PromptTemplate(input_variables=['adjective'], template = 'Tell me a {adjective} joke.')
one_input_prompt.format(adjective = 'funny')

'Tell me a funny joke.'

In [5]:
# multiple input variables

multiple_input_prompt = PromptTemplate(input_variables = ['adjective', 'context'], template = 'Tell me a {adjective} Joke about {context}.')
multiple_input_prompt.format(adjective = 'funny', context = 'chickens')

'Tell me a funny Joke about chickens.'

same can be done by creating PromptTemplate using from_template class method. langchain will automatically infer the input_variables based on the template passed.

In [6]:
template = 'Tell me a {adjective} Joke about {context}.'

prompt_template = PromptTemplate.from_template(template)
prompt_template.input_variables
prompt_template.format(adjective = 'funny', context = 'chickens')

'Tell me a funny Joke about chickens.'

## Chat prompt template

In [7]:
from langchain.prompts import (
    ChatPromptTemplate,
    PromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage,
)

<strong>MessagePromptTemplate</strong> is used to create a message template associate with role.

In [8]:
# we can also use from_template for same

template = 'You are a helpful assistant that translate {input_language} to {output_language}'
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_template = '{text}'
human_messege_prompt = HumanMessagePromptTemplate.from_template(human_template)

In [9]:
prompt = PromptTemplate(
    template = 'You are a helpful assistant that translate {input_language} to {output_language}',
    input_variables = ['input_language', 'output_language']
)

system_message_prompt_2 = SystemMessagePromptTemplate(prompt = prompt)

assert system_message_prompt == system_message_prompt_2

In [10]:
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_messege_prompt])

# get a chat completion from the formated messages
chat_prompt.format_prompt(input_language = "English", output_language = "Frence", text = 'Australia played very wellll.').to_messages()

[SystemMessage(content='You are a helpful assistant that translate English to Frence', additional_kwargs={}),
 HumanMessage(content='Australia played very wellll.', additional_kwargs={}, example=False)]

## Connecting to a Feature Store

we will use the popular open source feature store framework Feast.

### seting up the feast 

In [11]:
!pip install feast --q

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.6 which is incompatible.
apache-beam 2.46.0 requires protobuf<4,>3.12.2, but you have protobuf 4.23.3 which is incompatible.
google-api-core 1.33.2 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<4.0.0dev,>=3.19.5, but you have protobuf 4.23.3 which is incompatible.
google-cloud-artifact-registry 1.8.1 requires google-api-core[grpc]!=2.0.*,!=2.1.*,!=2.10.*,!=2.2.*,!=2.3.*,!=2.4.*,!=2.5.*,!=2.6.*,!=2.7.*,!=2.8.*,!=2.9.*,<3.0.0dev,>=1.34.0, but you have google-api-core 1.33.2 which is incompatible.
google-cloud-bigquery 2.34.4 requires protobuf<4.0.0dev,>=3.12.0, but you have protobuf 4.23.3 which is incompatible.
google-cloud-bigtable 1.7.3 requires protobuf<4.0.0dev, but you have protobuf 4.23.3 which 

In [21]:
!feast init my_project

/opt/conda/lib/python3.10/site-packages/feast/repo_operations.py:370: DeprecationWarning: The distutils package is deprecated and slated for removal in Python 3.12. Use setuptools or check PEP 632 for potential alternatives
  from distutils.dir_util import copy_tree

Creating a new Feast repository in /kaggle/working/my_project.



In [25]:
!cd /kaggle/working/my_project/feature_repo
!python /kaggle/working/my_project/feature_repo/test_workflow.py


Traceback (most recent call last):
  File "/kaggle/working/my_project/feature_repo/test_workflow.py", line 130, in <module>
    run_demo()
  File "/kaggle/working/my_project/feature_repo/test_workflow.py", line 11, in run_demo
    store = FeatureStore(repo_path=".")
  File "/opt/conda/lib/python3.10/site-packages/feast/usage.py", line 362, in wrapper
    raise exc.with_traceback(traceback)
  File "/opt/conda/lib/python3.10/site-packages/feast/usage.py", line 348, in wrapper
    return func(*args, **kwargs)
  File "/opt/conda/lib/python3.10/site-packages/feast/feature_store.py", line 154, in __init__
    self.config = load_repo_config(
  File "/opt/conda/lib/python3.10/site-packages/feast/repo_config.py", line 546, in load_repo_config
    with open(config_path) as f:
FileNotFoundError: [Errno 2] No such file or directory: 'feature_store.yaml'


In [13]:
from feast import FeatureStore
import pandas as pd
from datetime import datetime

entity_df = pd.DataFrame.from_dict({
    "driver_id": [1001, 1002, 1003, 1004],
    "event_timestamp": [
        datetime(2021, 4, 12, 10, 59, 42),
        datetime(2021, 4, 12, 8,  12, 10),
        datetime(2021, 4, 12, 16, 40, 26),
        datetime(2021, 4, 12, 15, 1 , 12)
    ]
})

store = FeatureStore(repo_path='/kaggle/working/myfeature_repo/feature_repo')

training_df = store.get_historical_features(
    entity_df=entity_df,
    features = [
        'driver_hourly_stats:conv_rate',
        'driver_hourly_stats:acc_rate',
        'driver_hourly_stats:avg_daily_trips'
    ],
).to_df()

print(training_df.head())

# Train model
# model = ml.fit(training_df)

FeatureViewNotFoundException: Feature view driver_hourly_stats does not exist

In [ ]:
from feast import FeatureStore

feast_rep_path = '/kaggle/working/myfeature_repo/feature_repo'
store = FeatureStore(repo_path = feast_rep_path)

Here we will set up a custom FeastPromptTemplate. This prompt template will take in a driver id, look up their stats, and format those stats into a prompt.

Note that the input to this prompt template is just driver_id, since that is the only user defined piece (all other variables are looked up inside the prompt template)

In [ ]:
from langchain.prompts import PromptTemplate, StringPromptTemplate

In [ ]:
template = """Given the driver's up to date stats, write them note relaying those stats to them.
If they have a conversation rate above .5, give them a compliment. Otherwise, make a silly joke about chickens at the end to make them feel better

Here are the drivers stats:
Conversation rate: {conv_rate}
Acceptance rate: {acc_rate}
Average Daily Trips: {avg_daily_trips}

Your response:"""

prompt = PromptTemplate.from_template(template)

In [ ]:
prompt

In [ ]:
class FeastPromptTemplate(StringPromptTemplate):
    def format(self, **kwargs) -> str:
        driver_id = kwargs.pop('driver_id')
        feature_vector = store.get_online_features(
            features = [
                'driver_hourly_stats:conv_rate',
                'driver_hourly_stats:acc_rate',
                'driver_hourly_stats:avg_daily_trips'
            ],
            entity_rows = [{'driver_id': driver_id}]
        ).to_dict()
        kwargs['conv_rate'] = feature_vector['conv_rate'][0]
        kwargs['acc_rate'] = feature_vector['acc_rate'][0]
        kwargs['avg_daily_trips'] = feature_vector['avg_daily_trips'][0]
        return prompt.format(**kwargs)

In [ ]:
prompt_template = FeastPromptTemplate(input_variables = ['driver_id'])

In [ ]:
prompt_template.format(driver_id = 1001)